<a href="https://colab.research.google.com/github/hyemiJ/data-dynamics/blob/main/python_practice/_12_%EC%B4%88%EB%B3%B4%EC%9E%90%EB%A5%BC_%EC%9C%84%ED%95%9C_python_200%EC%A0%9C_%EC%9B%B9%EC%84%9C%EB%B2%84_%EB%A1%9C%EA%B7%B8_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 웹 서버 로그 처리하기
- 우리는 인터넷 웹 브라우저를 통해 웹서버에 접속하여 다양한 컨텐츠를 이용한다.
- 웹서버는 웹브라우저에 대한 요청에 필요한 작업을 수행하고 HTML 페이지를 구성한 후 웹브라우저로 응답한다.
- 웹서버는 웹브라우저의 요청에 대한 응답 내용을 요약하여 접근 로그 파일에 기록한다.

- 아파치 웹서버는 오픈소스로 많은 웹사이트에서 사용하는 인기 있는 웹서버이다.
  - 아파치 웹서버 로그에 대한 자세한 내용은 링크를 참조한다.
  - https://httpd.apache.org/docs/2.2/ko/logs.html

- 공개 제공되는 웹 로그를 찾아 다운로드 한 후 실습을 진행한다.
  - https://github.com/elastic/examples/blob/master/Common%20Data%20Formats/apache_logs/apache_logs

In [9]:
# data 형태 보기
with open('apache_logs.txt') as f:
  logs = f.readlines()

  print(logs[0])

  log = logs[0]
  log = log.split()
  for i, l in enumerate(log) :
    print(f"인덱스 {i}: {l}")

83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "http://semicomplete.com/presentations/logstash-monitorama-2013/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"

인덱스 0: 83.149.9.216
인덱스 1: -
인덱스 2: -
인덱스 3: [17/May/2015:10:05:03
인덱스 4: +0000]
인덱스 5: "GET
인덱스 6: /presentations/logstash-monitorama-2013/images/kibana-search.png
인덱스 7: HTTP/1.1"
인덱스 8: 200
인덱스 9: 203023
인덱스 10: "http://semicomplete.com/presentations/logstash-monitorama-2013/"
인덱스 11: "Mozilla/5.0
인덱스 12: (Macintosh;
인덱스 13: Intel
인덱스 14: Mac
인덱스 15: OS
인덱스 16: X
인덱스 17: 10_9_1)
인덱스 18: AppleWebKit/537.36
인덱스 19: (KHTML,
인덱스 20: like
인덱스 21: Gecko)
인덱스 22: Chrome/32.0.1700.77
인덱스 23: Safari/537.36"


In [11]:
# 총 페이지 뷰 수 계산
pageviews = 0

with open('apache_logs.txt') as f:
  logs = f.readlines()
  for log in logs:
    log = log.split()
    status = log[8]
    if status == '200':
        pageviews +=1
print(pageviews)

9126


In [12]:
# 고유 방문자 수 계산
# 동일한 IP를 가진 클라이언트가 웹사이트에 10번을 접속해도 한사람이 방문되도록 계산.
visitip = []

with open('apache_logs.txt') as f:
  logs = f.readlines()
  for log in logs:
    log = log.split()
    ip = log[0] # 클라이언트 ip
    if ip not in visitip: # ip 리스트들에 없으면 추가, 있으면 패스.
      visitip.append(ip)
print('고유 방문자 수 :',len(visitip))

고유 방문자 수 : 1753


In [13]:
# 서비스 용량 계산
# 웹서버가 서비스한 콘텐츠의 총량을 계산한다.
KB = 1024
total_service = 0

with open('apache_logs.txt') as f:
  logs = f.readlines()
  for log in logs:
    log = log.split()
    servicebyte = log[9]
    if servicebyte.isdigit():
      total_service += int(servicebyte)

total_service = total_service/KB
print('서비스 용량 :',total_service,'KB')

서비스 용량 : 2682893.30078125 KB


In [18]:
# 사용자별 서비스 용량 계산
# IP 별 서비스 용량을 계산한다.
service = {}

with open('apache_logs.txt') as f:
  logs = f.readlines()
  for log in logs:
    log = log.split()
    ip = log[0]
    servicebyte = log[9]
    if servicebyte.isdigit():
      if ip in service:
        service[ip] += int(servicebyte)
      else:
        service[ip] = int(servicebyte)

result = sorted(service.items(), key=lambda x: x[1], reverse=True)

print('    사용자 IP    |  서비스 용량    ')
print('-'*30)
for ip, b in result[:10]:
  print(ip,' | ', b)

    사용자 IP    |  서비스 용량    
------------------------------
68.180.224.225  |  168132893
94.23.164.135  |  162949356
190.153.25.242  |  110134505
100.2.4.116  |  108670362
88.198.255.242  |  108632904
184.154.149.126  |  108613506
66.249.73.135  |  75500527
117.28.234.67  |  69210509
82.200.166.110  |  65259653
192.95.12.193  |  54377808
